In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc as misc 
import scipy.io as sio

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.style.use('dark_background')
plt.rcParams["font.family"] = "serif"

def eval_snr(x, x_est):
    if np.array_equal(x, x_est):
        return 0
    num = np.sqrt(np.sum(np.abs(x) ** 2))
    den = np.sqrt(np.sum(np.abs(x - x_est) ** 2))
    return round(20*np.log10(num/den), 2)


In [3]:
matlab_output = './../matlab_output/modified_Cai_results.mat'
# matlab_output = './../matlab_output/Cai_results.mat'

cai_resutls = sio.loadmat(matlab_output)
cai_resutls.keys()

dict_keys(['__header__', '__version__', '__globals__', 'LCI_10_h_bound', 'LCI_10_l_bound', 'LCI_20_h_bound', 'LCI_20_l_bound', 'LCI_30_h_bound', 'LCI_30_l_bound', 'delta', 'dirty_im', 'f_x_map', 'g_x_map', 'gridSize', 'im', 'im_name', 'nlevel', 'sol_ana', 'stepSize', 'tau_alpha'])


In [3]:

cai_resutls['tau_alpha']

array([[67981.57752119]])

In [4]:
savefig_dir = './../figs/'
save_name = '{:s}_256_wavelet_modified_matlab'.format(str(cai_resutls['im_name'][0]))
# save_name = '{:s}_256_wavelet_matlab'.format(str(cai_resutls['im_name'][0]))


# Load variables
x_init = np.abs(cai_resutls['dirty_im'])
x = np.abs(cai_resutls['im'])
x_hat_np = cai_resutls['sol_ana']
N = cai_resutls['im'].size

print(
    '\nf(x_map): ', cai_resutls['f_x_map'][0,0],
    '\ng(x_map): ', cai_resutls['g_x_map'][0,0],
    '\ntau_alpha*np.sqrt(N): ', cai_resutls['tau_alpha'][0,0]-N,
    '\nN: ', N,
)


f(x_map):  417200.0308836563 
g(x_map):  1774143.3659090172 
tau_alpha*np.sqrt(N):  2445.5775211890286 
N:  65536


In [5]:
images = [x, x_init, x_hat_np, x-np.abs(x_hat_np)]
labels = ["Truth", "Dirty", "Reconstruction", "Residual (x - x^hat)"]
ground_truth = x

# Plot results

fig, axs = plt.subplots(1,4, figsize=(24,6), dpi=200)
for i in range(4):
    im = axs[i].imshow(images[i], cmap='cubehelix', vmax=np.nanmax(images[i]), vmin=np.nanmin(images[i]))
    divider = make_axes_locatable(axs[i])
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im, cax=cax, orientation='vertical')
    if i > 0:   
        stats_str = '\n(PSNR: {},\n SNR: {}, SSIM: {})'.format(
            round(psnr(ground_truth, images[i],data_range=ground_truth.max()-ground_truth.min()), 2),
            round(eval_snr(x, images[i]), 2),
            round(ssim(ground_truth, images[i]), 2),
            )
        labels[i] += stats_str
        print(labels[i])
    axs[i].set_title(labels[i], fontsize=16)
    axs[i].axis('off')
plt.savefig('{:s}{:s}_MAP.pdf'.format(savefig_dir, save_name))
plt.close()


Dirty
(PSNR: 27.45,
 SNR: 5.01, SSIM: 0.58)
Reconstruction
(PSNR: 36.73,
 SNR: 14.29, SSIM: 0.96)
Residual (x - x^hat)
(PSNR: 23.08,
 SNR: 0.64, SSIM: 0.82)


In [6]:


for it in range(cai_resutls['gridSize'].shape[1]):
    superpix_size = cai_resutls['gridSize'][0,it]
    print('superpix_size: ', superpix_size)

    error_p = cai_resutls['LCI_{:d}_h_bound'.format(superpix_size)]
    error_m = cai_resutls['LCI_{:d}_l_bound'.format(superpix_size)]
    error_length = error_p - error_m

    vmin = np.min((x, x_init, x_hat_np))
    vmax = np.max((x, x_init, x_hat_np))
    # err_vmax= 0.6
    cmap='cubehelix'

    plt.figure(figsize=(28,12))
    plt.subplot(241)
    plt.imshow(x, cmap=cmap, vmin=vmin, vmax=vmax);plt.colorbar()
    plt.title('Ground truth')
    plt.subplot(242)
    plt.imshow(x_init, cmap=cmap, vmin=vmin, vmax=vmax);plt.colorbar()
    plt.title('Dirty')
    plt.subplot(243)
    plt.imshow(x_hat_np, cmap=cmap, vmin=vmin, vmax=vmax);plt.colorbar()
    plt.title('MAP estimator')
    plt.subplot(244)
    plt.imshow(x - x_hat_np, cmap=cmap);plt.colorbar()
    plt.title('Ground truth - MAP estimator')
    plt.subplot(245)
    plt.imshow(error_length, cmap=cmap, vmin=0, vmax=0.3);plt.colorbar()
    plt.title('LCI (max={:.5f})\n (mean={:.5f})'.format(
        np.max(error_length), np.mean(error_length))
    )
    plt.subplot(246)
    plt.imshow(error_length - np.mean(error_length), cmap=cmap);plt.colorbar()
    plt.title('LCI - <LCI>')
    plt.savefig(
        '{:s}{:s}_pixSize_{:d}.pdf'.format(savefig_dir, save_name, superpix_size)
    )
    plt.close()

    
    it+=1

superpix_size:  10
superpix_size:  20
superpix_size:  30


## Compare function evaluations


In [23]:
numpy_results = np.load('/Users/tliaudat/Documents/postdoc/github/large-scale-UQ/debug/output/M31_256_wavelet-db8_4_reg_10.0_results.npy', allow_pickle=True)[()]

numpy_cai_results = np.load('/Users/tliaudat/Documents/postdoc/github/large-scale-UQ/debug/output/M31_256_wavelet-db8_4_reg_10.0_Cai_sol_results.npy', allow_pickle=True)[()]

CRR_results = np.load('/Users/tliaudat/Documents/postdoc/github/large-scale-UQ/debug/output/M31_256_CRR_mu20_lmbd100_results.npy', allow_pickle=True)[()]


In [24]:
print('cai_resutls[f_x_map]: {:.1f}'.format(cai_resutls['f_x_map'][0,0]))
print('cai_resutls[g_x_map]: {:.1f}'.format(cai_resutls['g_x_map'][0,0]))
print('cai_resutls[tau_alpha]: {:.1f}\n'.format(cai_resutls['tau_alpha'][0,0]))

print('numpy_cai_results[hpd_results][f_xmap]: {:.1f}'.format(numpy_cai_results['hpd_results']['f_xmap']))
print('numpy_cai_results[hpd_results][g_xmap]: {:.1f}'.format(numpy_cai_results['hpd_results']['g_xmap']))
print('numpy_cai_results[hpd_results][h_alpha_N]: {:.1f}\n'.format(numpy_cai_results['hpd_results']['h_alpha_N']))

print('numpy_results[hpd_results][f_xmap]: {:.1f}'.format(numpy_results['hpd_results']['f_xmap']))
print('numpy_results[hpd_results][g_xmap]: {:.1f}'.format(numpy_results['hpd_results']['g_xmap']))
print('numpy_results[hpd_results][h_alpha_N]: {:.1f}\n'.format(numpy_results['hpd_results']['h_alpha_N']))

print('CRR_results[hpd_results][f_xmap]: {:.1f}'.format(CRR_results['hpd_results']['f_xmap']))
print('CRR_results[hpd_results][g_xmap]: {:.1f}'.format(CRR_results['hpd_results']['g_xmap']))
print('CRR_results[hpd_results][h_alpha_N]: {:.1f}\n'.format(CRR_results['hpd_results']['h_alpha_N']))



cai_resutls[f_x_map]: 417200.0
cai_resutls[g_x_map]: 1774143.4
cai_resutls[tau_alpha]: 67981.6

numpy_cai_results[hpd_results][f_xmap]: 426906.7
numpy_cai_results[hpd_results][g_xmap]: 9242.1
numpy_cai_results[hpd_results][h_alpha_N]: 67981.6

numpy_results[hpd_results][f_xmap]: 56577.5
numpy_results[hpd_results][g_xmap]: 16377.7
numpy_results[hpd_results][h_alpha_N]: 67981.6

CRR_results[hpd_results][f_xmap]: 0.0
CRR_results[hpd_results][g_xmap]: 74.9
CRR_results[hpd_results][h_alpha_N]: 67981.6



In [28]:

CRR_results.keys()

dict_keys(['x_map', 'opt_params', 'hpd_results', 'error_p_arr', 'error_m_arr', 'computing_time', 'superpix_sizes', 'LCI_params'])

In [33]:
cp_x_map = np.copy(numpy_results['x_map'])
save_dic = {
    'cp_x_map': cp_x_map,
}
save_path = '/Users/tliaudat/Documents/postdoc/github/large-scale-UQ/debug/matlab_output/'
sio.savemat(save_path+'np_xmap.mat', save_dic)